In [70]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from IPython.display import display, Markdown, Latex
import sympy
from sympy import *
from sympy.physics.units import *

def display_var(var, value, unit=None, preface=None):
    display(Markdown("$\\text{{ {}${} = {:.3f} \\text{{ {} }} $ }} $".format(preface or "", var, value, unit or "")))

# Pi Echo

_Introduction_

## Background


## Aproach
### Simulation

Simulating a system like this is actually rather straight forward. The goal of each simulation is to determine the performance of each set of parameters. The key parameters are the sample frequency, $f_s$, which affects the limits of all other parameters and performance characteristics. Primarily, it determines the highest frequency which can be detected by the system, in accordance with the Nyquist Sampling Theorm $f_s > 2f_{max}$. In our application, $f_{max}$ is going to be the emitted frequency doppler shifted by the maximum speed we want to detect. In general, the doppler shift will be very small compared to the frequency itself, so we'll generalize the constraint to be

$$ f_s > 2f_0 $$

where f_0 is the emitted frequency. $f_0$ itself affects the performance characteristics a marginal amount. The doppler effect is defined as

$$ f_{doppler} = {c \over c+v} f_0 $$

When a standard Discrete Fourier Transform is used to find frequencies, a higher $f_0$ will result in a proportional shift in $f_{doppler}$ as a result, which will span more frequency bins in the transform.

In [96]:
c = 343 # m/s

# Returns the frequency shifted by the doppler effect for a given velocity 'v'
def doppler(f0, v):
    return c / (c + v) * f0

display_var("f_0 - f_{doppler}", 5000 - doppler(5000, 1), "Hz", " for $f_0 = 5000$ Hz: ")
display_var("f_0 - f_{doppler}", 22000 - doppler(22000, 1), "Hz", " for $f_0 = 22000$ Hz: ")

$\text{  for $f_0 = 5000$ Hz: $f_0 - f_{doppler} = 14.535 \text{ Hz } $ } $

$\text{  for $f_0 = 22000$ Hz: $f_0 - f_{doppler} = 63.953 \text{ Hz } $ } $

The next important parameter is $n_s$, which is the number of samples used for each Fourier Transform. First and foremost, $n_s$ determines the amount of time over which samples are gathered for one calculation, $T_{FFT}$. 

$$ T_{FFT} = T_s n_s = {n_s \over f_s} $$

In [90]:
# Returns the sample period given a sample frequency
def sample_period(f_s):
    return 1 / f_s

# Returns the time (in seconds) over which one transform would be calculated
def fft_period(T_s, n):
    return T_s * n

display_var("T_s", sample_period(44100) * 1000, "ms")
display_var("T_{FFT}", fft_period(sample_period(44100), 8192) * 1000, "ms")

$\text{ $T_s = 0.023 \text{ ms } $ } $

$\text{ $T_{FFT} = 185.760 \text{ ms } $ } $

The number of samples also defines the number of frequency bins that will be in the resulting Transform $2n_{bins} = n_s$. The more frequency bins there are, the higher the resolution of the spectrum, $f_{bin}$ since those bins are distributed over the Nyquist frequency $f_s \over 2$.

In [97]:
# Returns the number of hertz per frequency bin in the fft
def freq_per_bin(T_s, n):
    return 1 / fft_period(T_s, n)

''' For a given set of parameters, returns the resolution with which
    the system will be able to detect the doppler effect.
    Essentially by resolution we mean the number of bins that span
    the range of velocities we want to detect.
'''
def resolution(T_s, n, f0, vmax):
    return abs(int((doppler(f0, vmax) - f0) / freq_per_bin(T_s, n)))

display_var("f_{bin}", freq_per_bin(sample_period(44100), 8192), "Hz")
display_var("\\text{resolution}", resolution(sample_period(44100), 8192, 22000, 1))

$\text{ $f_{bin} = 5.383 \text{ Hz } $ } $

$\text{ $\text{resolution} = 11.000 \text{  } $ } $

In general, $n_{s}$ will also be chosen as a power of 2 as FFT algorithms run most efficently for those sample counts.